# VacationPy

---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data


In [50]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [51]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,springbok,-29.6643,17.8865,15.64,71,7,2.30,ZA,1707334311
1,1,nueva esperanza,15.2667,-87.6000,24.79,50,48,1.83,HN,1707334311
2,2,bilibino,68.0546,166.4372,-35.41,100,99,0.40,RU,1707334311
3,3,blackmans bay,-43.0167,147.3167,15.07,75,100,0.27,AU,1707334312
4,4,mulchen,-37.7167,-72.2333,28.92,22,89,3.56,CL,1707334312


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [52]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    color="City",
    alpha=0.5,
    s="Humidity",
    scale=0.75
)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition


In [53]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= 20)
    & (city_data_df["Country"] == "US")
    & (city_data_df["Humidity"] > 49)
    & (city_data_df["Humidity"] < 61),
    :,
]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,albany,42.6001,-73.9662,6.16,53,64,0.89,US,1707334202
150,150,ketchikan,55.3422,-131.6461,2.99,60,75,5.14,US,1707334330
201,201,broome,42.2506,-75.8330,7.88,52,0,1.34,US,1707334340
393,393,lompoc,34.6391,-120.4579,14.32,53,75,5.66,US,1707334278
415,415,rawlins,41.7911,-107.2387,4.64,60,100,8.75,US,1707334172
417,417,westport,41.1415,-73.3579,8.83,53,32,0.45,US,1707334373
442,442,rhinelander,45.6366,-89.4121,8.18,56,0,2.06,US,1707334376
563,563,marion,39.7684,-86.1555,11.76,51,100,0.00,US,1707334341


### Step 3: Create a new DataFrame called `hotel_df`.


In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:, [1, 8, 2, 3, 5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,albany,US,42.6001,-73.9662,53,
150,ketchikan,US,55.3422,-131.6461,60,
201,broome,US,42.2506,-75.8330,52,
393,lompoc,US,34.6391,-120.4579,53,
415,rawlins,US,41.7911,-107.2387,60,
417,westport,US,41.1415,-73.3579,53,
442,rhinelander,US,45.6366,-89.4121,56,
563,marion,US,39.7684,-86.1555,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.


In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key,
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"][
            "name"
        ]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(
        f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}"
    )

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
broome - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
rawlins - nearest hotel: Quality Inn Rawlins I-80
westport - nearest hotel: Norwalk Inn and Conference Center
rhinelander - nearest hotel: Days Inn & Suites
marion - nearest hotel: Hilton Garden Inn Indianapolis Downtown


,City,Country,Lat,Lng,Humidity,Hotel Name
7,albany,US,42.6001,-73.9662,53,No hotel found
150,ketchikan,US,55.3422,-131.6461,60,Cape Fox Lodge
201,broome,US,42.2506,-75.8330,52,No hotel found
393,lompoc,US,34.6391,-120.4579,53,Embassy Suites by Hilton Lompoc Central Coast
415,rawlins,US,41.7911,-107.2387,60,Quality Inn Rawlins I-80
417,westport,US,41.1415,-73.3579,53,Norwalk Inn and Conference Center
442,rhinelander,US,45.6366,-89.4121,56,Days Inn & Suites
563,marion,US,39.7684,-86.1555,51,Hilton Garden Inn Indianapolis Downtown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.


In [56]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    color="City",
    alpha=0.5,
    hover=True,
    hover_cols="all",
    use_index=False,
    s="Humidity",
    scale=2,
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)